In [3]:
!pip install torch torchvision numpy wget

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 2.5 MB/s eta 0:00:0000:0100:12m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:0000:0100:06m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 3.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 2.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 3.0 MB/s eta 0:00:0000:0100:05m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 3.1 MB/s eta 0:00:0000:0100:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 703.3 kB/s eta 0:00:0000:0100:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 2.4 MB/s

In [1]:
import os
import zipfile
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# 1. SETUP & REPRODUCIBILITY
# Setting seeds for reproducibility as requested in the homework
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

# Ensure deterministic behavior for CuDNN
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
# Check if data.zip exists, if not - download it
if not os.path.exists('data.zip'):
    print("Downloading data...")
    !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip

# Unzip the data into the current directory
if not os.path.exists('data'):
    print("Unzipping data...")
    with zipfile.ZipFile('data.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
    print("Data extracted!")
else:
    print("Data folder already exists.")

--2025-12-02 19:26:46--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T20%3A08%3A08Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T19%3A08%3A02Z&ske=2025-12-02T20%3A08%3A08Z&sks=b&skv=2018-11-09&sig=ZZk%2F2h7gJBu8j79HtBbhFMWlrhkrwK9uU4pZJBKDepk%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDcwNTQwNywibmJmIjoxNzY0NzAzNjA3LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5i

In [3]:
class HairNet(nn.Module):
    def __init__(self):
        super(HairNet, self).__init__()
        # Input shape: (3, 200, 200)
        
        # Convolutional Layer
        # in_channels=3 (RGB), out_channels=32, kernel=3x3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=0, stride=1)
        self.relu = nn.ReLU()
        
        # Max Pooling Layer
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        
        # Shape calculation:
        # 1. Conv2d: (W - F + 2P)/S + 1 
        #    -> (200 - 3 + 0)/1 + 1 = 198
        #    Output shape: (32, 198, 198)
        # 2. MaxPool2d: 198 / 2 = 99
        #    Output shape: (32, 99, 99)
        # 3. Flatten size: 32 * 99 * 99 = 313,632
        
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1) # Output layer (1 neuron for binary classification)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        # Flatten the output for the fully connected layer
        x = torch.flatten(x, 1) 
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Initialize model to count parameters
model = HairNet().to(device)

# Question 2: Total number of parameters
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}") 
# Expected answer usually around 20073473

Total parameters: 20073473


In [4]:
def train_model(model, train_loader, val_loader, optimizer, criterion, epochs, history=None):
    """
    Main training loop. 
    If 'history' is provided, it appends new metrics to it.
    """
    if history is None:
        history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}
    
    for epoch in range(epochs):
        # --- Training Phase ---
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            # Reshape labels to (batch_size, 1) and ensure float type
            labels = labels.float().unsqueeze(1)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            
            # Apply sigmoid for binary classification accuracy
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
            
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_train / total_train
        
        history['loss'].append(epoch_loss)
        history['acc'].append(epoch_acc)
        
        # --- Validation Phase ---
        model.eval()
        val_running_loss = 0.0
        correct_val = 0
        total_val = 0
        
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                labels = labels.float().unsqueeze(1)
                
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_running_loss += loss.item() * images.size(0)
                predicted = (torch.sigmoid(outputs) > 0.5).float()
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
                
        val_epoch_loss = val_running_loss / len(val_loader.dataset)
        val_epoch_acc = correct_val / total_val
        
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_epoch_acc)
        
        print(f"Epoch {epoch+1}/{epochs} | "
              f"Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f} | "
              f"Val Loss: {val_epoch_loss:.4f} | Val Acc: {val_epoch_acc:.4f}")
        
    return history

In [5]:
print("--- Phase 1: Training WITHOUT Augmentation ---")

# 1. Re-initialize the model
model = HairNet().to(device)

# 2. Define Loss and Optimizer (Question 1: BCEWithLogitsLoss)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# 3. Define Transforms (Basic normalization only)
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 4. Prepare Datasets and Loaders
train_dataset = datasets.ImageFolder('./data/train', transform=train_transforms)
val_dataset = datasets.ImageFolder('./data/test', transform=train_transforms) # Test folder used for validation

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False, num_workers=2)

# 5. Run Training
history_phase1 = train_model(model, train_loader, val_loader, optimizer, criterion, epochs=10)

# 6. Calculate Answers for Q3 & Q4
median_acc = np.median(history_phase1['acc'])
std_loss = np.std(history_phase1['loss'])

print("\n" + "="*40)
print(f"Question 3 (Median Train Accuracy): {median_acc:.4f}")
print(f"Question 4 (Standard Deviation of Train Loss): {std_loss:.4f}")
print("="*40)

--- Phase 1: Training WITHOUT Augmentation ---
Epoch 1/10 | Loss: 0.6654 | Acc: 0.6262 | Val Loss: 0.6102 | Val Acc: 0.6617
Epoch 2/10 | Loss: 0.5479 | Acc: 0.7087 | Val Loss: 0.6303 | Val Acc: 0.6468
Epoch 3/10 | Loss: 0.4858 | Acc: 0.7638 | Val Loss: 0.6991 | Val Acc: 0.5970
Epoch 4/10 | Loss: 0.4806 | Acc: 0.7650 | Val Loss: 0.6097 | Val Acc: 0.7015
Epoch 5/10 | Loss: 0.4403 | Acc: 0.7863 | Val Loss: 0.6215 | Val Acc: 0.6517
Epoch 6/10 | Loss: 0.3493 | Acc: 0.8500 | Val Loss: 0.6453 | Val Acc: 0.6716
Epoch 7/10 | Loss: 0.3005 | Acc: 0.8775 | Val Loss: 0.6954 | Val Acc: 0.6816
Epoch 8/10 | Loss: 0.2746 | Acc: 0.8950 | Val Loss: 0.6396 | Val Acc: 0.7264
Epoch 9/10 | Loss: 0.2163 | Acc: 0.9187 | Val Loss: 0.6716 | Val Acc: 0.7065
Epoch 10/10 | Loss: 0.1466 | Acc: 0.9563 | Val Loss: 0.7745 | Val Acc: 0.7114

Question 3 (Median Train Accuracy): 0.8181
Question 4 (Standard Deviation of Train Loss): 0.1527


In [6]:
print("\n--- Phase 2: Training WITH Augmentation ---")

# 1. Define NEW Transforms with Augmentation
train_transforms_aug = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)), # Ensure final size is correct
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 2. Re-create Training Dataset and Loader with new transforms
# Note: We do NOT re-create the model. We continue training the existing one.
train_dataset_aug = datasets.ImageFolder('./data/train', transform=train_transforms_aug)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True, num_workers=2)

# 3. Continue Training
# We pass 'history_phase1' so the new metrics are appended to it
history_phase2 = train_model(model, train_loader_aug, val_loader, optimizer, criterion, epochs=10, history=history_phase1)

# 4. Calculate Answers for Q5 & Q6
# We need to look at the metrics from the SECOND phase (epochs 11-20)
# Since 'history' is a single list now containing 20 epochs, we slice the last 10 entries.

aug_val_losses = history_phase2['val_loss'][10:]  # Epochs 11-20
aug_val_accs = history_phase2['val_acc'][10:]    # Epochs 11-20

# Question 5: Mean of test (val) loss for ALL epochs trained with augmentations
mean_test_loss_aug = np.mean(aug_val_losses)

# Question 6: Average of test (val) accuracy for the LAST 5 epochs (epochs 16-20)
# In our slice [10:], indices 5,6,7,8,9 correspond to the last 5 epochs.
avg_test_acc_last_5 = np.mean(aug_val_accs[5:])

print("\n" + "="*40)
print(f"Question 5 (Mean Validation Loss with Augmentation): {mean_test_loss_aug:.4f}")
print(f"Question 6 (Avg Validation Accuracy, last 5 epochs): {avg_test_acc_last_5:.4f}")
print("="*40)


--- Phase 2: Training WITH Augmentation ---
Epoch 1/10 | Loss: 0.6820 | Acc: 0.6587 | Val Loss: 0.6043 | Val Acc: 0.6816
Epoch 2/10 | Loss: 0.5393 | Acc: 0.7113 | Val Loss: 0.7068 | Val Acc: 0.6468
Epoch 3/10 | Loss: 0.5190 | Acc: 0.7400 | Val Loss: 0.5641 | Val Acc: 0.6965
Epoch 4/10 | Loss: 0.5407 | Acc: 0.7125 | Val Loss: 0.5927 | Val Acc: 0.7114
Epoch 5/10 | Loss: 0.4818 | Acc: 0.7662 | Val Loss: 0.5380 | Val Acc: 0.7313
Epoch 6/10 | Loss: 0.4880 | Acc: 0.7450 | Val Loss: 0.5159 | Val Acc: 0.7264
Epoch 7/10 | Loss: 0.4934 | Acc: 0.7550 | Val Loss: 0.5079 | Val Acc: 0.7662
Epoch 8/10 | Loss: 0.4860 | Acc: 0.7738 | Val Loss: 0.4994 | Val Acc: 0.7811
Epoch 9/10 | Loss: 0.4443 | Acc: 0.7825 | Val Loss: 0.6461 | Val Acc: 0.6468
Epoch 10/10 | Loss: 0.4532 | Acc: 0.7900 | Val Loss: 0.5438 | Val Acc: 0.7264

Question 5 (Mean Validation Loss with Augmentation): 0.5719
Question 6 (Avg Validation Accuracy, last 5 epochs): 0.7294
